In [75]:
import pandas as pd, numpy as np

In [76]:
train = pd.read_csv(r'C:\Users\Anjana Tiha\OneDrive - The University of Memphis\Programming\data\house-prices-advanced-regression-techniques\train.csv')
test = pd.read_csv(r'C:\Users\Anjana Tiha\OneDrive - The University of Memphis\Programming\data\house-prices-advanced-regression-techniques\train.csv')

In [77]:
data = train+test

In [78]:
#feature_cols = ['Pclass','Sex','Age','SibSp', 'Parch','Fare','Cabin','Embarked']

target_col = 'SalePrice'

target = data[target_col]

data.drop(columns=target_col)

features = data

features.describe()
target.describe()


rating_cols = ['ExterQual' 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
rating_score = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0}
rating_cols2 = {'BsmtFinType1', 'BsmtFinType2'}
rating_score2 = {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0}

ignore_cols = ['BsmtExposure']

In [79]:
from sklearn import preprocessing

print(data.info())
print("\n----------------------\n")
# print(data.isnull().sum(axis=1))   

features.replace(-np.Inf, np.nan)
features.replace(np.Inf, np.nan)


dtype_grp = features.columns.to_series().groupby(data.dtypes).groups

for dtype in dtype_grp:
    print(dtype_grp[dtype])

for dtype in dtype_grp:
    if dtype=='int64' :
        for col in dtype_grp[dtype]:
            features[col] = features[col].fillna(0)
            features[col]=preprocessing.normalize([features[col]], norm='l2').flatten()
    elif dtype=='float64' :
        for col in dtype_grp[dtype]:
            features[col] = features[col].fillna(0.0)
            features[col]=preprocessing.normalize([features[col]], norm='l2').flatten()
    elif dtype=='object' :
        for col in dtype_grp[dtype]:
            if col in rating_cols:
                features[col].replace(rating_score, regex=True, inplace=True)
            elif col in rating_cols2:
                features[col].replace(rating_score2, regex=True, inplace=True)
            else:
                if col in ignore_cols:
                    continue
                try:
                    le = preprocessing.LabelEncoder()
                    enc = preprocessing.OneHotEncoder()
                    le.fit(features[col].astype(str))
                    label_enc_feature_val = le.transform(features[col].astype(str))
                    features[col] = label_enc_feature_val
                    
#                     enc.fit([label_enc_feature_val])  
#                     hot_enc_feature_val = enc.transform([label_enc_feature_val])

#                     print(hot_enc_feature_val)

#                     features.join(hot_enc_feature_val)
#                     features.drop(columns=col)
                except ValueError:
                    print(col)
                    print("Oops!  That was no valid number.  Try again...")
                

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [80]:
print(features.info())
X = features.values
Y = target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null float64
MSSubClass       1460 non-null float64
MSZoning         1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null float64
Street           1460 non-null int64
Alley            1460 non-null int64
LotShape         1460 non-null int64
LandContour      1460 non-null int64
Utilities        1460 non-null int64
LotConfig        1460 non-null int64
LandSlope        1460 non-null int64
Neighborhood     1460 non-null int64
Condition1       1460 non-null int64
Condition2       1460 non-null int64
BldgType         1460 non-null int64
HouseStyle       1460 non-null int64
OverallQual      1460 non-null float64
OverallCond      1460 non-null float64
YearBuilt        1460 non-null float64
YearRemodAdd     1460 non-null float64
RoofStyle        1460 non-null int64
RoofMatl         1460 non-null int64
Exterior1st      1460 non-

In [81]:
from sklearn.model_selection import KFold

from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
    
def model_evaluation(X, Y, splitter, model, report, details):
    explained_variance_score = 0
    mean_absolute_error = 0
    mean_squared_error = 0
    mean_squared_log_error = 0
    median_absolute_error = 0
    r2_score = 0
    i=0
    if report:
        print("*"*50, " START ", "*"*50)
        print("Spliter Description:")
        print(splitter)
        print("-"*100, "\n")
        print("Model Description:")
        print(model)
        print("-"*100,"\n")
    
    for train_index, test_index in splitter.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        # model fitting
        model.fit(X_train, y_train)
        
        # prediction
        predict = model.predict(X_test)

        # evaluation scores
        explained_variance_score_temp = explained_variance_score(y_test, predict)
        mean_absolute_error_temp = mean_absolute_error(y_test, predict)
        mean_squared_error_temp = mean_squared_error(y_test, predict)
        mean_squared_log_error_temp = mean_squared_log_error(y_test, predict)
        median_absolute_error_temp = median_absolute_error(y_test, predict)
        r2_score_temp = r2_score(y_test, predict)
        
        explained_variance_score = explained_variance_score + explained_variance_score_temp
        mean_absolute_error = mean_absolute_error + mean_absolute_error_temp
        mean_squared_error = mean_squared_error + mean_squared_error_temp
        mean_squared_log_error = mean_squared_log_error + mean_squared_log_error_temp
        median_absolute_error= median_absolute_error + median_absolute_error_temp
        r2_score= r2_score + r2_score_temp
        
        if details:
            print("*"*25,  " ITERATION - ", i+1, "*"*25)
            #print("TRAIN:", train_index, "TEST:", test_index)
            print("explained_variance_score_temp: ", explained_variance_score_temp)
            print("mean_absolute_error_temp: ", mean_absolute_error_temp)
            print("mean_squared_error_temp: ", mean_squared_error_temp)
            print("mean_squared_log_error_temp: ", mean_squared_log_error_temp)
            print("median_absolute_error_temp: ", median_absolute_error_temp)
            print("r2_score_temp: ", r2_score_temp)
            
#             print("-"*35)
#             print(metrics.classification_report(y_test, predict))
#             print("-"*35)
#             print("confusion Matrix:\n\n", metrics.confusion_matrix(y_test, predict))
#             print("-"*35)
            print("\n")
        
        i+=1
    
    split_num = splitter.get_n_splits()
    
    explained_variance_score = explained_variance_score/split_num
    mean_absolute_error = mean_absolute_error/split_num
    mean_squared_error = mean_squared_error/split_num
    mean_squared_log_error = mean_squared_log_error/split_num
    median_absolute_error = median_absolute_error/split_num
    r2_score = r2_score/split_num
    
    if report:
        print("*"*50, " Average For", i+1, " Folds", "*"*50)
        print("\n")
        print("Average explained_variance_score: ", explained_variance_score)
        print("Average mean_absolute_error: ", mean_absolute_error)
        print("Average mean_squared_error: ", mean_squared_error)
        print("Average mean_squared_log_error:", mean_squared_log_error)
        print("Average median_absolute_error:", median_absolute_error)
        print("Average r2_score:", r2_score)
        print("\n")
        print("*"*50, " END ", "*"*50)
    
    
    
    return explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score


In [82]:
from sklearn.model_selection import train_test_split

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomTreesEmbedding, RandomForestRegressor, VotingClassifier)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.isotonic import IsotonicRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ARDRegression, LinearRegression, LogisticRegression, LogisticRegressionCV, logistic_regression_path, HuberRegressor, PassiveAggressiveRegressor, RandomizedLogisticRegression, RANSACRegressor, SGDRegressor, TheilSenRegressor
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB 
from sklearn.neighbors import KDTree, KNeighborsRegressor, NearestNeighbors, RadiusNeighborsRegressor
from sklearn.neural_network import BernoulliRBM, MLPRegressor
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor



kf = KFold()


regressors = {
    "AdaBoostRegressor": AdaBoostRegressor(),
    "ARDRegression": ARDRegression(),
    "BaggingRegressor": BaggingRegressor(),
#     "BernoulliRBM": BernoulliRBM(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "ExtraTreesRegressor": ExtraTreesRegressor(),
    "ExtraTreeRegressor": ExtraTreeRegressor(),
#     "GaussianMixture": GaussianMixture(),
#     "GaussianNB": GaussianNB(),
    "GaussianProcessRegressor": GaussianProcessRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "HuberRegressor": HuberRegressor(),
    "IsotonicRegression": IsotonicRegression(),
    "KernelRidge": KernelRidge(),
#     "KDTree": KDTree(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "LinearRegression": LinearRegression(), 
    "LogisticRegression": LogisticRegression(),
    "LogisticRegressionCV": LogisticRegressionCV(),
#     "logistic_regression_path": logistic_regression_path(),
    "LinearSVR": LinearSVR(),
    "MLPRegressor": MLPRegressor(),
#     "MultinomialNB": MultinomialNB(),
    "NuSVR": NuSVR(),
    "PassiveAggressiveRegressor": PassiveAggressiveRegressor(),
#     "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    "RadiusNeighborsRegressor": RadiusNeighborsRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "RandomizedLogisticRegression": RandomizedLogisticRegression(),
    "RANSACRegressor": RANSACRegressor(),
    "SGDRegressor": SGDRegressor(),
    "SVR": SVR(),
    "TheilSenRegressor": TheilSenRegressor(),
}
    
    
splitter = kf 
report = 1
details = 1


evaluation = {}

for name in regressors:
    evaluation_temp = []
    
    explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score = model_evaluation(X, Y, splitter, regressor[name], report, details)
    evaluation_temp.append(explained_variance_score)
    evaluation_temp.append(mean_absolute_error)
    evaluation_temp.append(mean_squared_error)
    evaluation_temp.append(mean_squared_log_error)
    evaluation_temp.append(median_absolute_error)
    evaluation_temp.append(r2_score)
    evaluation[name] = evaluation_temp
    

rows_list = []
for name in evaluation:
    rows_list.append([name]+evaluation[name])
                           
evaluation_pd = pd.DataFrame(rows_list, columns=['explained_variance_score',  'mean_absolute_error', 'mean_squared_error', 'mean_squared_log_error', 'median_absolute_error', 'r2_score']) 
evaluation_pd

**************************************************  START  **************************************************
Spliter Description:
KFold(n_splits=3, random_state=None, shuffle=False)
---------------------------------------------------------------------------------------------------- 

Model Description:
AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)
---------------------------------------------------------------------------------------------------- 



C:\Users\Anjana Tiha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


ValueError: could not convert string to float: 'NoNo'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure


figure(num=None, figsize=(14, 6), dpi=250)

labels= ['accuracy', 'precision', 'recall', 'f1']
ax = plt.subplot(111)

for n in range(0,4):
    plt.plot([name for name in evaluation],[evaluation[name][n] for name in evaluation], label = labels[n])

leg = plt.legend(loc='best', ncol=2, mode="expand", shadow=True, fancybox=True)
plt.xticks(rotation=45)
# leg.get_frame().set_alpha(0.5)
plt.legend()
ax.tick_params(labelsize='large', width=5)
ax.grid(True, linestyle='-.')

plt.tight_layout()
plt.xlabel('x label')
plt.ylabel('y label')

plt.title("TITLE")
plt.show()



